In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

## Scraping the Player data for the required Years from Basketball Reference (https://www.basketball-reference.com/) 


In [ ]:
# NBA season we will be analyzing
year = 2015# URL page we will scraping (see image above)
url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)# this is the HTML from the given URL
html = urlopen(url)
soup = BeautifulSoup(html)

# use findALL() to get the column headers
soup.findAll('tr', limit=2)# use getText()to extract the text we need into a list
headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]# exclude the first column as we will not need the ranking order from Basketball Reference for the analysis
headers = headers[1:]
# avoid the first header row
rows = soup.findAll('tr')[1:]
player_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

## Select Relevant columns to populate the DataFrame

In [ ]:
cols = ['Year','Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

In [ ]:
stats = pd.DataFrame(player_stats, columns = headers)
stats["Year"] = year
stats = stats[cols]
#stats = stats.append(stats)

#### Save DataFrame to CSV

In [ ]:
stats.to_csv("Stats 2015-2019")

# Scraping the Team data for the Relevant Years

### Creating a list of team to Scrape for to avoid scraping Teams that are no longer playing

In [11]:
Teams = ['LAL', 'GSW', 'PHO', 'BOS', 'CHI', 'IND', 'SAC',
        'HOU', 'SAS', 'POR', 'MIL', 'DET', 'PHI', 
        'UTA', 'DEN', 'ATL', 'CLE', 'NYK', 'DAL', 
        'LAC', 'MIA', 'ORL', 'MIN', 'TOR', 'WAS', 
        'MEM', 'CHA', 'OKC', 'BRK', 'NOP']

In [13]:
Tm_Soups = []
for i in Teams :
    url = "https://www.basketball-reference.com/teams/" + str(i) + "/"
    #.format(t)
    html = urlopen(url)
    soup = BeautifulSoup(html,"lxml")
    soup.findAll('tr', limit=2)
    Tm_Soups.append(soup)

In [16]:
headers = [th.getText() for th in Tm_Soups[0].findAll('tr', limit=2)[0].findAll('th')]
headers = headers[1:]

## Pre - Processing the data before creating the DataFrame

In [60]:
Team_name = []
Season = []
Wins = []
Year_tm = []
Tm = []
a=0
for l in Tm_Soups:  
    y = 2019  
    rows = l.findAll('tr')
    team_stats = [[td.getText() for td in rows[i].findAll('td') or rows[i].findAll('th')]
                for i in range(len(rows))]
    
    for t in range(0,len(team_stats)):
        if team_stats[t][1] != "Lg":
            Team_name.append(team_stats[t][1])
            Wins.append(team_stats[t][3])
            Year_tm.append(y)
            y = y-1
            Tm.append(Teams[a])
    a+=1    


In [61]:
Final_teams = []
for i in range(0,len(Team_name)):
    Final_teams.append([Year_tm[i],Tm[i],Team_name[i],Wins[i]])
cols_tm = ["Year","Tm","Team_Name","Wins"] 

In [63]:
Team_df = pd.DataFrame(Final_teams)
Team_df.columns = cols_tm

In [64]:
Team_df.head()

,Year,Tm,Team_Name,Wins
0,2019,LAL,Los Angeles Lakers*,19
1,2018,LAL,Los Angeles Lakers,45
2,2017,LAL,Los Angeles Lakers,47
3,2016,LAL,Los Angeles Lakers,56
4,2015,LAL,Los Angeles Lakers,65


#### Save Team data to CSV

In [65]:
Team_df.to_csv("Input/Team_stats")